In [1]:
import sys
import os

from xml.dom.minidom import parse
from nltk.tokenize import word_tokenize

In [2]:
DATA_PATH = '/home/bscuser/Documentos/second_term/AHLT/ahlt/data/Train/DrugBank'

In [8]:
# -------- classify_token ----------
# -- check if a token is a drug, and of which type
suffixes = ['azole', 'amine', 'idine', 'mycin']


# cat datapath.xml | xmllint -format - | grep entity | awk '{print $(NF -1),$NF}'

def classify_token(txt):
    # Baseline approach
    # Todo: Add more rules
    if txt.isupper():
        return True, "brand"
    elif txt[-5:] in suffixes:
        return True, "drug"
    else:
        return False, ""


# --------- tokenize sentence -----------
# -- Tokenize sentence, returning tokens and span offsets

def tokenize(txt):
    offset = 0
    tks = []
    # word_tokenize splits words, taking into account punctuations, numbers, etc.
    for t in word_tokenize(txt):
        # keep track of the position where each token should appear, and
        # store that information with the token
        offset = txt.find(t, offset)
        tks.append((t, offset, offset + len(t) - 1))
        offset += len(t)

    # tks is a list of triples (word,start,end)
    return tks


# --------- Entity extractor ----------- 
# -- Extract drug entities from given text and return them as
# -- a list of dictionaries with keys "offset", "text", and "type"

def extract_entities(stext):
    # convert the sentece to a list of tokens
    tokens = tokenize(stext)

    # for each token, check whether it is a drug name or not
    result = []
    for t in tokens:
        tokenTxt = t[0]
        (is_drug, tk_type) = classify_token(tokenTxt)

        # Todo: Change this part to accumulate drugs that appear together
        if is_drug:
            drug_start = t[1]
            drug_end = t[2]
            drug_type = tk_type
            e = {"offset": str(drug_start) + "-" + str(drug_end),
                 "text": stext[drug_start:drug_end],
                 "type": drug_type}
            result.append(e)

    return result

def parse_sentences(file_path):
    tree = parse(file_path)
    sentences = tree.getElementsByTagName("sentence")
    parsed_sentences = {}
    for s in sentences:
        sid = s.attributes["id"].value  # get sentence id
        stext = s.attributes["text"].value  # get sentence text
        tokenized = tokenize(stext)
        parsed_sentences[sid] = tokenized
    return parsed_sentences


# process each file in directory
for f in os.listdir(DATA_PATH):
    s = parse_sentences(os.path.join(DATA_PATH,f))
    print(s)
    print()

{'DDI-DrugBank.d85.s0': [('The', 0, 2), ('use', 4, 6), ('of', 8, 9), ('NSAIDs', 11, 16), ('in', 18, 19), ('patients', 21, 28), ('who', 30, 32), ('are', 34, 36), ('receiving', 38, 46), ('ACE', 48, 50), ('inhibitors', 52, 61), ('may', 63, 65), ('potentiate', 67, 76), ('renal', 78, 82), ('disease', 84, 90), ('states', 92, 97), ('.', 98, 98)], 'DDI-DrugBank.d85.s1': [('In', 0, 1), ('vitro', 3, 7), ('studies', 9, 15), ('have', 17, 20), ('shown', 22, 26), ('that', 28, 31), ('naproxen', 33, 40), ('anion', 42, 46), (',', 47, 47), ('because', 49, 55), ('of', 57, 58), ('its', 60, 62), ('affinity', 64, 71), ('for', 73, 75), ('protein', 77, 83), (',', 84, 84), ('may', 86, 88), ('displace', 90, 97), ('from', 99, 102), ('their', 104, 108), ('binding', 110, 116), ('sites', 118, 122), ('other', 124, 128), ('drugs', 130, 134), ('which', 136, 140), ('are', 142, 144), ('also', 146, 149), ('albumin-bound', 151, 163), ('.', 165, 165)], 'DDI-DrugBank.d85.s2': [('Theoretically', 0, 12), (',', 13, 13), ('the'

{'DDI-DrugBank.d186.s0': [('An', 0, 1), ('encephalopathic', 3, 17), ('syndrome', 19, 26), ('(', 28, 28), ('characterized', 29, 41), ('by', 43, 44), ('weakness', 46, 53), (',', 54, 54), ('lethargy', 56, 63), (',', 64, 64), ('fever', 66, 70), (',', 71, 71), ('tremulousness', 73, 85), ('and', 87, 89), ('confusion', 91, 99), (',', 100, 100), ('extrapyramidal', 102, 115), ('symptoms', 117, 124), (',', 125, 125), ('leukocytosis', 127, 138), (',', 139, 139), ('elevated', 141, 148), ('serum', 150, 154), ('enzymes', 156, 162), (',', 163, 163), ('BUN', 165, 167), (',', 168, 168), ('and', 170, 172), ('FBS', 174, 176), (')', 177, 177), ('followed', 179, 186), ('by', 188, 189), ('irreversible', 191, 202), ('brain', 204, 208), ('damage', 210, 215), ('has', 217, 219), ('occurred', 221, 228), ('in', 230, 231), ('a', 233, 233), ('few', 235, 237), ('patients', 239, 246), ('treated', 248, 254), ('with', 256, 259), ('lithium', 261, 267), ('plus', 269, 272), ('HALDOL', 274, 279), ('.', 280, 280)], 'DDI-Dru

{'DDI-DrugBank.d29.s0': [('Esomeprazole', 0, 11), ('is', 13, 14), ('extensively', 16, 26), ('metabolized', 28, 38), ('in', 40, 41), ('the', 43, 45), ('liver', 47, 51), ('by', 53, 54), ('CYP2C19', 56, 62), ('and', 64, 66), ('CYP3A4', 68, 73), ('.', 74, 74)], 'DDI-DrugBank.d29.s1': [('In', 0, 1), ('vitro', 3, 7), ('and', 9, 11), ('in', 13, 14), ('vivo', 16, 19), ('studies', 21, 27), ('have', 29, 32), ('shown', 34, 38), ('that', 40, 43), ('esomeprazole', 45, 56), ('is', 58, 59), ('not', 61, 63), ('likely', 65, 70), ('to', 72, 73), ('inhibit', 75, 81), ('CYPs', 83, 86), ('1A2', 88, 90), (',', 91, 91), ('2A6', 93, 95), (',', 96, 96), ('2C9', 98, 100), (',', 101, 101), ('2D6', 103, 105), (',', 106, 106), ('2E1', 108, 110), ('and', 112, 114), ('3A4', 116, 118), ('.', 119, 119)], 'DDI-DrugBank.d29.s2': [('No', 0, 1), ('clinically', 3, 12), ('relevant', 14, 21), ('interactions', 23, 34), ('with', 36, 39), ('drugs', 41, 45), ('metabolized', 47, 57), ('by', 59, 60), ('these', 62, 66), ('CYP', 68,

{'DDI-DrugBank.d270.s0': [('Nevirapine', 0, 9), ('is', 11, 12), ('principally', 14, 24), ('metabolized', 26, 36), ('by', 38, 39), ('the', 41, 43), ('liver', 45, 49), ('via', 51, 53), ('the', 55, 57), ('cytochrome', 59, 68), ('P450', 70, 73), ('isoenzymes', 75, 84), (',', 85, 85), ('3A4', 87, 89), ('and', 91, 93), ('2B6', 95, 97), ('.', 98, 98)], 'DDI-DrugBank.d270.s1': [('Nevirapine', 0, 9), ('is', 11, 12), ('known', 14, 18), ('to', 20, 21), ('be', 23, 24), ('an', 26, 27), ('inducer', 29, 35), ('of', 37, 38), ('these', 40, 44), ('enzymes', 46, 52), ('.', 53, 53)], 'DDI-DrugBank.d270.s2': [('As', 0, 1), ('a', 3, 3), ('result', 5, 10), (',', 11, 11), ('drugs', 13, 17), ('that', 19, 22), ('are', 24, 26), ('metabolized', 28, 38), ('by', 40, 41), ('these', 43, 47), ('enzyme', 49, 54), ('systems', 56, 62), ('may', 64, 66), ('have', 68, 71), ('lower', 73, 77), ('than', 79, 82), ('expected', 84, 91), ('plasma', 93, 98), ('levels', 100, 105), ('when', 107, 110), ('coadministered', 112, 125), ('

{'DDI-DrugBank.d94.s0': [('Clinically', 0, 9), ('meaningful', 11, 20), ('drug', 22, 25), ('interactions', 27, 38), ('have', 40, 43), ('occurred', 45, 52), ('with', 54, 57), ('concomitant', 59, 69), ('medications', 71, 81), ('and', 83, 85), ('include', 87, 93), (',', 94, 94), ('but', 96, 98), ('are', 100, 102), ('not', 104, 106), ('limited', 108, 114), ('to', 116, 117), ('the', 119, 121), ('following', 123, 131), (':', 132, 132), ('Agents', 134, 139), ('Highly', 141, 146), ('Bound', 148, 152), ('to', 154, 155), ('Plasma', 157, 162), ('Protein', 164, 170), ('Carbamazepine', 172, 184), ('is', 186, 187), ('not', 189, 191), ('highly', 193, 198), ('bound', 200, 204), ('to', 206, 207), ('plasma', 209, 214), ('proteins', 216, 223), (';', 224, 224)], 'DDI-DrugBank.d94.s1': [('therefore', 0, 8), (',', 9, 9), ('administration', 11, 24), ('of', 26, 27), ('EQUETROTM', 29, 37), ('to', 39, 40), ('a', 42, 42), ('patient', 44, 50), ('taking', 52, 57), ('another', 59, 65), ('drug', 67, 70), ('that', 72,

...

{'DDI-DrugBank.d296.s0': [('Diphenhydramine', 0, 14), ('hydrochloride', 16, 28), ('has', 30, 32), ('additive', 34, 41), ('effects', 43, 49), ('with', 51, 54), ('alcohol', 56, 62), ('and', 64, 66), ('other', 68, 72), ('CNS', 74, 76), ('depressants', 78, 88), ('(', 90, 90), ('hypnotics', 91, 99), (',', 100, 100), ('sedatives', 102, 110), (',', 111, 111), ('tranquilizers', 113, 125), (',', 126, 126), ('etc', 128, 130), (')', 131, 131), ('.', 132, 132)], 'DDI-DrugBank.d296.s1': [('MAO', 0, 2), ('inhibitors', 4, 13), ('prolong', 15, 21), ('and', 23, 25), ('intensify', 27, 35), ('the', 37, 39), ('anticholinergic', 41, 55), ('(', 57, 57), ('drying', 58, 63), (')', 64, 64), ('effects', 66, 72), ('of', 74, 75), ('antihistamines', 77, 90), ('.', 91, 91)]}

{'DDI-DrugBank.d110.s0': [('The', 0, 2), ('administration', 4, 17), ('of', 19, 20), ('local', 22, 26), ('anesthetic', 28, 37), ('solutions', 39, 47), ('containing', 49, 58), ('epinephrine', 60, 70), ('or', 72, 73), ('norepinephrine', 75, 88), 

{'DDI-DrugBank.d314.s0': [('Aminoglutethimide', 0, 16), (':', 17, 17), ('Aminoglutethimide', 19, 35), ('may', 37, 39), ('diminish', 41, 48), ('adrenal', 50, 56), ('suppression', 58, 68), ('by', 70, 71), ('corticosteroids', 73, 87), ('.', 88, 88)], 'DDI-DrugBank.d314.s1': [('Amphotericin', 0, 11), ('B', 13, 13), ('injection', 15, 23), ('and', 25, 27), ('potassium-depleting', 29, 47), ('agents', 49, 54), (':', 55, 55), ('When', 57, 60), ('corticosteroids', 62, 76), ('are', 78, 80), ('administered', 82, 93), ('concomitantly', 95, 107), ('with', 109, 112), ('potassium-depleting', 114, 132), ('agents', 134, 139), ('(', 141, 141), ('e.g.', 142, 145), (',', 146, 146), ('amphotericin', 148, 159), ('B', 161, 161), (',', 162, 162), ('diuretics', 164, 172), (')', 173, 173), (',', 174, 174), ('patients', 176, 183), ('should', 185, 190), ('be', 192, 193), ('observed', 195, 202), ('closely', 204, 210), ('for', 212, 214), ('development', 216, 226), ('of', 228, 229), ('hypokalemia', 231, 241), ('.', 2

{'DDI-DrugBank.d233.s0': [('Use', 0, 2), ('with', 4, 7), ('Allopurinol', 9, 19), (':', 20, 20), ('The', 22, 24), ('principal', 26, 34), ('pathway', 36, 42), ('for', 44, 46), ('detoxification', 48, 61), ('of', 63, 64), ('azathioprine', 66, 77), ('is', 79, 80), ('inhibited', 82, 90), ('by', 92, 93), ('allopurinol', 95, 105), ('.', 106, 106)], 'DDI-DrugBank.d233.s1': [('Patients', 0, 7), ('receiving', 9, 17), ('azathioprine', 19, 30), ('and', 32, 34), ('allopurinol', 36, 46), ('concomitantly', 48, 60), ('should', 62, 67), ('have', 69, 72), ('a', 74, 74), ('dose', 76, 79), ('reduction', 81, 89), ('of', 91, 92), ('azathioprine', 94, 105), (',', 106, 106), ('to', 108, 109), ('approximately', 111, 123), ('1/3', 125, 127), ('to', 129, 130), ('1/4', 132, 134), ('the', 136, 138), ('usual', 140, 144), ('dose', 146, 149), ('.', 150, 150)], 'DDI-DrugBank.d233.s2': [('Use', 0, 2), ('with', 4, 7), ('Other', 9, 13), ('Agents', 15, 20), ('Affecting', 22, 30), ('Myelopoesis', 32, 42), (':', 43, 43), ('D

{'DDI-DrugBank.d92.s0': [('No', 0, 1), ('well-known', 3, 12), ('drug', 14, 17), ('interactions', 19, 30), ('with', 32, 35), ('glutamic', 37, 44), ('acid', 46, 49)]}

{'DDI-DrugBank.d398.s0': [('The', 0, 2), ('use', 4, 6), ('of', 8, 9), ('codeine', 11, 17), ('may', 19, 21), ('result', 23, 28), ('in', 30, 31), ('additive', 33, 40), ('CNS', 42, 44), ('depressant', 46, 55), ('effects', 57, 63), ('when', 65, 68), ('coadministered', 70, 83), ('with', 85, 88), ('alcohol', 90, 96), (',', 97, 97), ('antihistamines', 99, 112), (',', 113, 113), ('psychotropics', 115, 127), ('or', 129, 130), ('other', 132, 136), ('drugs', 138, 142), ('that', 144, 147), ('produce', 149, 155), ('CNS', 157, 159), ('depression', 161, 170), ('.', 171, 171)], 'DDI-DrugBank.d398.s1': [('Serious', 0, 6), ('toxicity', 8, 15), ('may', 17, 19), ('result', 21, 26), ('if', 28, 29), ('dextromethorphan', 31, 46), ('is', 48, 49), ('coadministered', 51, 64), ('with', 66, 69), ('monoamine', 71, 79), ('oxidase', 81, 87), ('inhibitor

{'DDI-DrugBank.d521.s0': [('Many', 0, 3), ('other', 5, 9), ('medicines', 11, 19), ('may', 21, 23), ('increase', 25, 32), ('or', 34, 35), ('decrease', 37, 44), ('the', 46, 48), ('effects', 50, 56), ('of', 58, 59), ('glimepiride', 61, 71), ('or', 73, 74), ('affect', 76, 81), ('your', 83, 86), ('condition', 88, 96), ('.', 97, 97)], 'DDI-DrugBank.d521.s1': [('Before', 0, 5), ('taking', 7, 12), ('glimepiride', 14, 24), (',', 25, 25), ('tell', 27, 30), ('your', 32, 35), ('doctor', 37, 42), ('if', 44, 45), ('you', 47, 49), ('are', 51, 53), ('taking', 55, 60), ('any', 62, 64), ('of', 66, 67), ('the', 69, 71), ('following', 73, 81), ('medicines', 83, 91), (':', 92, 92), ('-', 94, 94), ('aspirin', 96, 102), ('or', 104, 105), ('another', 107, 113), ('salicylate', 115, 124), ('such', 126, 129), ('as', 131, 132), ('magnesium/choline', 134, 150), ('salicylate', 152, 161), ('(', 163, 163), ('Trilisate', 164, 172), (')', 173, 173), (',', 174, 174), ('salsalate', 176, 184), ('(', 186, 186), ('Disalcid'